<a href="https://colab.research.google.com/github/mhassack/challenges/blob/master/2024_12_29_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install langchain_google_genai langchain_google_community langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12


In [3]:
import os
import logging

from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_community import BigQueryVectorStore
from langchain.prompts import PromptTemplate, StringPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

from langchain_core.runnables import RunnableLambda

In [4]:
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.DEBUG)


In [10]:
RUNNING_IN_COLAB = True

if RUNNING_IN_COLAB:
    logger.debug("Running in Colab")
    from google.colab import userdata
    GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
    from google.colab import auth
    auth.authenticate_user()
    from IPython.display import HTML, display

    def set_css():
      display(HTML('''
      <style>
        pre {
            white-space: pre-wrap;
        }
      </style>
      '''))
    get_ipython().events.register('pre_run_cell', set_css)
else:
    logger.debug("Loading dotenv")
    from dotenv import load_dotenv
    load_dotenv()  # take environment variables from .env.
    GEMINI_API_KEY = os.environ['GEMINI_API_KEY']

# Define the name of the GCP project
GOOGLE_CLOUD_PROJECT = "ai-experiments-445909"


In [6]:

# Create our embeddings object
logger.debug("Creating the embeddings object")
embedding = GoogleGenerativeAIEmbeddings(
    model="models/text-embedding-004", google_api_key=GEMINI_API_KEY
)


In [7]:

# Define the BQ bits
BIGQUERY_DATASET_ID = "rag_experiments"
BIGQUERY_TABLE_ID = "test_embeddings"
BIGQUERY_LOCATION = "europe-west2"

# Create the BQ vector store
logger.debug("Create the BigQuery Vector Store")
store = BigQueryVectorStore(
    project_id=GOOGLE_CLOUD_PROJECT,
    dataset_name=BIGQUERY_DATASET_ID,
    table_name=BIGQUERY_TABLE_ID,
    embedding=embedding,
    location=BIGQUERY_LOCATION
)

# Retrieve and generate using the relevant snippets of the blog.
logger.debug("Geetting the BQ store as a retreiver")
retriever = store.as_retriever()


INFO:langchain_google_community.bq_storage_vectorstores._base:BigQuery table ai-experiments-445909.rag_experiments.test_embeddings initialized/validated as persistent storage. Access via BigQuery console:
 https://console.cloud.google.com/bigquery?project=ai-experiments-445909&ws=!1m5!1m4!4m3!1sai-experiments-445909!2srag_experiments!3stest_embeddings


In [12]:
# Create the LLM interface
logger.debug("Create the LLM object")
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    google_api_key=GEMINI_API_KEY
)



In [13]:


# Define the non RAG chain
non_rag_chain = (
    llm
    | StrOutputParser())

# Query the LLM using the non RAG chain
response = non_rag_chain.invoke("In the context of Asterisk, tell me what IAX is")
print(f"Non RAG response: {response}")



Non RAG response: IAX stands for **Inter-Asterisk eXchange**. It's a networking protocol specifically designed for Voice over IP (VoIP) communication, created by Mark Corrigan, the founder of Asterisk.  While it can be used with other VoIP systems, it's tightly integrated with and commonly associated with Asterisk.

Here's a breakdown of key characteristics and benefits of IAX:

* **Single Port Operation:** IAX uses a single UDP port (port 4569 by default) for both signaling and media streams. This simplifies firewall configuration and NAT traversal compared to SIP, which typically uses separate ports for signaling and different media streams (RTP).

* **Lower Bandwidth Consumption:** IAX is designed for efficiency, particularly in low-bandwidth environments. It uses less overhead than SIP, making it suitable for situations with limited network resources.  This efficiency is achieved through binary encoding and session multiplexing.

* **NAT Traversal:** IAX excels at NAT traversal, of

In [8]:

# Create the prompt
logger.debug("Getting the LLM prompt")
template = """
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know.
Question: {question}
Context: {context}
Answer:
"""

ragPrompt = PromptTemplate.from_template(template)



In [15]:

def inspect(state):
    print(state)
    return state

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Create the RAG chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | ragPrompt
    | RunnableLambda(inspect)
    | llm
    | StrOutputParser()
)

# Query the LLM via the RAG chain
response = rag_chain.invoke("In the context of Asterisk, tell me what IAX is")
print(f"\n\nRAG response: {response}")

logger.debug("DONE")



text="\nYou are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know.\nQuestion: In the context of Asterisk, tell me what IAX is \nContext: to note that IAX is not at all limited to Asterisk. The standard is open for anyone to\n\nseveral hardware vendors. IAX is a transport protocol (much like SIP) that uses a single\n\nhave helped make IAX one of the easiest protocols (if not the easiest) to implement in\n\nproper pronunciation is in fact “eeks.”‡ IAX is an open protocol, meaning that anyone\n\nIf your ITSP is offering IAX2 termination, there is a strong chance it is running Asterisk; \nAnswer:\n"


RAG response: IAX (pronounced "eeks") is a transport protocol, similar to SIP, used in Asterisk and other systems. It uses a single port and is considered one of the easiest protocols to implement.  It's an open protocol, meaning anyone can use it.

